Import the necessary libraries

In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import IterableDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split


Define hyperparameters

In [4]:
TRAIN_PROPORTION = 0.7
BATCH_SIZE = 64
EPOCHS = 3
LEARNING_RATE = 0.05

Extract data from CSV file and prepare train and test data

Create custom dataset from CSV file

In [5]:
class ReviewDataset(Dataset):
    def __init__(self, filepath, transform=None, target_transform=None):
        self.data = pd.read_csv(filepath)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data.loc[idx]
        review = entry["Review"]
        label = entry["Rating"]
        if self.transform:
            review = self.transform(review)
        if self.target_transform:
            label = self.target_transform(label)
        return review, label

Create datasets and create corresponding DataLoader objects

In [6]:
all_data = ReviewDataset("data.csv")
train_len = round(TRAIN_PROPORTION * len(all_data))
train_data, test_data = random_split(all_data, [train_len, len(all_data) - train_len], generator=torch.Generator().manual_seed(42))

Define neural network

In [7]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

Create pipelines to generate embeddings
Create a collate function to combine text entries in the original data batch into a single Tensor

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def create_review_iter():
    for review,_ in train_data:
        #print(review)
        yield tokenizer(review)


vocab = build_vocab_from_iterator(create_review_iter(), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def collate_batch(batch):
   # print("SHOWING SIZES")
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         #print(_text)
        # print(_label)
         #print("$$$$$$$$$$$$$$$$$$$$$$")
         #print("??????????????????????")
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int)
         text_list.append(processed_text)
         #print(processed_text.size(0))
         offsets.append(0)
    label_list = torch.tensor(label_list, dtype=torch.int)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    print("XXXXXXXXXXXXXXXXXX")
    print(label_list)
    print("XXXXXXXXXXXXXXXXXX")
    text_list = torch.cat(text_list)
    offsets[-1] = 0
    return text_list.to(device), label_list.to(device), offsets.to(device)

#create DataLoaders
dataloader = DataLoader(all_data, batch_size=BATCH_SIZE, shuffle=True)

Create model

In [9]:
model = TextClassificationModel(len(vocab), 64, 5).to(device)

Define the loss function and optimizer

In [10]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    x = 0
    vals = {
        "0":torch.tensor([0,0,0,0,0]),
        "1":torch.tensor([1,0,0,0,0]),
        "2":torch.tensor([0,1,0,0,0]),
        "3":torch.tensor([0,0,1,0,0]),
        "4":torch.tensor([0,0,0,1,0]),
        "5":torch.tensor([0,0,0,0,1]),
    }
    for idx, (text, label, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        print(text)
        print("////////////////////////")
        print(vals[str(label[x].item())])
       # print(offsets)
        predicted_label = model(text, offsets)
        print("**************")
        print(predicted_label)
        print("**************")
        loss = criterion(predicted_label, label) #(((predicted_label - vals[str(label[x].item())])**2)/5).sum()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            #print('| epoch {:3d} | {:5d}/{:5d} batches '
             #     '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
             #                                 total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()
        x += 1

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (text, label, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

Carry out the training and testing

In [12]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
#EPOCHS = 10 # epoch

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
num_train = int(len(train_data) * 0.95)
split_train_, split_valid_ = \
    random_split(train_data, [num_train, len(train_data) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
   # print('-' * 59)
    #print('| end of epoch {:3d} | time: {:5.2f}s | '
     #     'valid accuracy {:8.3f} '.format(epoch,
      #                                     time.time() - epoch_start_time,
       #                                    accu_val))
    #print('-' * 59)

XXXXXXXXXXXXXXXXXX
tensor([3, 4, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4,
        3, 4, 4, 4, 0, 3, 4, 4, 3, 4, 4, 4, 4, 3, 4, 1, 4, 3, 2, 2, 3, 2, 3, 4,
        4, 4, 4, 4, 4, 3, 4, 4, 2, 3, 4, 0, 3, 3, 4, 3], dtype=torch.int32)
XXXXXXXXXXXXXXXXXX
tensor([  40,   20,   15,  ...,   47, 1874,    1], dtype=torch.int32)
////////////////////////
tensor([0, 0, 1, 0, 0])
**************
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.000

RuntimeError: expected scalar type Long but found Int